In [1]:
import os
import scenic
from scenic.simulators.gta.nusc_query_api import NuscQueryAPI
import json

# query = NuscQueryAPI(version = 'v1.0-trainval', dataroot='/Users/edwardkim/Desktop/nuScenes_data')
query = NuscQueryAPI(version = 'v1.0-mini', dataroot='/Users/edwardkim/Desktop/v1.0-mini')

Loading NuScenes tables for version v1.0-mini...
23 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
Done loading in 0.6 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.


In [ ]:
import pickle
with open("/Users/edwardkim/Desktop/Scenic_bigdata_query/data/valid_numCar_trafficFlow_assumption/boston_seaport_valid_traffic_flow.pickle", 'rb') as f:
        image_filenames = pickle.load(f)

directory = "/Users/edwardkim/Desktop/nuScenes_data/samples/boston_seaport"
# image_filenames = [file for file in os.listdir(directory) if file.endswith('.jpg')]
# image_filenames = [file for file in boston.keys() if file.endswith('.jpg')]

sc_boston_seaport = '../examples/gta/test_boston_seaport.sc' # great map
# sc_singapore_onenorth = 'examples/gta/test_singapore_onenorth.sc' # great map

# construct scenario
scenario_boston_seaport = scenic.scenarioFromFile(sc_boston_seaport)
# scenario_singapore_onenorth = scenic.scenarioFromFile(sc_singapore_onenorth)

# How many cars are there in the scenario?
NUM_CAR = 4 # excluding ego

# create a dictionary of scenarios depending on maps:
scenario_list = {'boston-seaport'           : scenario_boston_seaport}
#                  'singapore-onenorth'       : scenario_singapore_onenorth}

In [2]:
import matplotlib.pyplot as plt
from shapely.geometry import Polygon
from descartes.patch import PolygonPatch
from imutils.perspective import order_points
from scenic.core.regions import regionFromShapelyObject
import math

def convert_deg_to_rad(degree):
    '''convert NuScenes degree label to Scenic's radians -- there is -90 offset '''
    return ((degree + 360) % 360) * math.pi / 180 - (math.pi/ 2) 

def checkUserRequirements(scenario_copy, label, num_car):
    # extract ego's NuScenes label values
    egoCar = label['EgoCar']
    ego_position = egoCar['position']
    ego_heading = convert_deg_to_rad(egoCar['heading'])

    # condition ego obj's position and heading to NuScene's label values
    scenario_copy.egoObject.position.conditionTo(Vector(ego_position[0], ego_position[1]))
    scenario_copy.egoObject.heading.conditionTo(Constant(ego_heading))
    scenario_copy.dependencies[0].position.conditionTo(Vector(ego_position[0], ego_position[1]))
    scenario_copy.dependencies[0].heading.conditionTo(Constant(ego_heading))

    # extract NuScenes' non-ego car's labels
    otherVehicles = label['Vehicles']

    for i in range(num_car-1):
        scenic_car = scenario_copy.dependencies[i+1]
        nuscenes_car = otherVehicles[i+1]
        nusc_car_position = nuscenes_car['position']
        nusc_car_heading  = convert_deg_to_rad(nuscenes_car['heading'])

        # condition on nuscenes' position and heading values
        scenario_copy.dependencies[i+1].position.conditionTo(Vector(nusc_car_position[0], nusc_car_position[1]))
        scenario_copy.dependencies[i+1].heading.conditionTo(Constant(nusc_car_heading))
        scenario_copy.objects[i+1].position.conditionTo(Vector(nusc_car_position[0], nusc_car_position[1]))
        scenario_copy.objects[i+1].heading.conditionTo(Constant(nusc_car_heading))
        
    result = scenario_copy.checkForRequirements(maxIterations = 1)
    valid  = result[0]
    scene  = result[1]
    
    if not valid:
        return [False, None]
    
#     # this checks for user-defined hard-constraints
#     if isinstance(scene, Scene):
#         imgs_in_scenario.append(img_name)
#         print("the scene SATISFIES USER REQUIREMENTS!")
#         scene.show(zoom = True)
    return [True, scene]

def checkPosHeading(scenario, label, num_car):
    
    scenario_copy = copy.deepcopy(scenario)
    
    # extract ego's NuScenes label values
    egoCar = label['EgoCar']
    ego_position = egoCar['position']
    ego_heading = convert_deg_to_rad(egoCar['heading'])

    # condition ego obj's position and heading to NuScene's label values
    scenario_copy.egoObject.position.conditionTo(Vector(ego_position[0], ego_position[1]))
    scenario_copy.egoObject.heading.conditionTo(Constant(ego_heading))
    scenario_copy.dependencies[0].position.conditionTo(Vector(ego_position[0], ego_position[1]))
    scenario_copy.dependencies[0].heading.conditionTo(Constant(ego_heading))

    # extract NuScenes' non-ego car's labels
    otherVehicles = label['Vehicles']

    for i in reversed(range(num_car-1)):

        # recover the original distribution from conditioned values
        scenario_copy.dependencies[i+1].position._conditioned = scenario_copy.dependencies[i+1].position._original
        scenario_copy.dependencies[i+1].heading._conditioned  = scenario_copy.dependencies[i+1].heading._original
        
        # extract corresponding vehicle's nuScenes label
        nuscenes_car = otherVehicles[i+1]
        nusc_car_position = nuscenes_car['position']
        nusc_car_heading  = ((nuscenes_car['heading'] - 90 + 360) % 360) # need to be in degrees
        
        roadDirection = label['traffic_flow']((nusc_car_position[0], nusc_car_position[1]))
        if isinstance(roadDirection, str):
            print("roadDirection of Car"+str(i+1)+": ", label['traffic_flow']((nusc_car_position[0], nusc_car_position[1])))
            return False

        # Check for Position's Validity
        scenario_copy.dependencies = scenario_copy.dependencies[:i+2] # this counts up to (i+1)th
        scenario_copy.objects      = scenario_copy.objects[:i+2]
        pos_valid = checkPosition(scenario_copy, obj_index = i+1, position= nusc_car_position, sampleNum = 200)

        if not pos_valid:
            return False
            
        # Conditioning on Position, check for heading validity
        scenario_copy.dependencies[i+1].position.conditionTo(Vector(nusc_car_position[0], nusc_car_position[1]))
        heading_valid = checkHeading(scenario_copy, obj_index= i+1, heading_angle = nusc_car_heading, sampleNum = 200)
        
        if not heading_valid:
            return False
        
    return True


def computeDist(point1, point2):
    return math.sqrt(math.pow(point1[0]-point2[0],2) + math.pow(point1[1]-point2[1],2))

def approximateContourPoints(dictionary, x_max, x_min, reverse = False, coarseness = 5):
    step = (x_max - x_min)/coarseness
    division_points = [(x_min + i*step) for i in range(coarseness)] + [x_max]
    d = sorted(dictionary.keys())
    points, furthest_point = [], None
    index = 0
    
    for k in d:
        if division_points[index] <= k[0] and k[0] <= division_points[index+1]:
            if furthest_point == None:
                furthest_point = k
            else:
                if dictionary[furthest_point] <= dictionary[k]:
                    furthest_point = k
        else:
            index += 1
            if furthest_point != None:
                points.append(furthest_point)
            furthest_point = None
            
    if reverse:
        points.reverse()
        return points
    
    return points

def checkPosition(scenario, obj_index, position, sampleNum = 10):
    print("CheckPosition: Car" + str(obj_index) + " located at ", position)
    x_pos = []
    y_pos = []
    points = []
    
    for i in range(sampleNum):
        scene = scenario.sampleForNuScenes()
        obj = scene.objects[obj_index]
        points.append((obj.position.x, obj.position.y))
        x_pos.append(obj.position.x)
        y_pos.append(obj.position.y)
        
    max_x = x_pos.index(max(x_pos))
    min_x = x_pos.index(min(x_pos))
    max_y = y_pos.index(max(y_pos))
    min_y = y_pos.index(min(y_pos))
    mean_x_val = (x_pos[max_x] + x_pos[min_x])/2
    mean_y_val = (y_pos[max_y] + y_pos[min_y])/2
    
    slope = (points[max_x][1] - points[min_x][1]) / (points[max_x][0] - points[min_x][0])
    bias  = points[max_x][1] - slope * points[max_x][0] # b = y - slope * x
    
    leftmost  = points[min_x]
    rightmost = points[max_x]
    upperhalf, bottomhalf = {},{}
    
    for p in points:
        distance = computeDist(leftmost, p) + computeDist(rightmost, p)
        if p[1] - slope*p[0] > bias :
            upperhalf[p] = distance
        else:
            bottomhalf[p] = distance
            
    upper_points = approximateContourPoints(upperhalf, x_pos[max_x], x_pos[min_x], reverse=False, coarseness = 11)
    bottom_points = approximateContourPoints(bottomhalf, x_pos[max_x], x_pos[min_x], reverse=True, coarseness = 11)

    points = [leftmost] + upper_points + [rightmost] + bottom_points
    
    polygon = Polygon(points)
    scenic_polygon = regionFromShapelyObject(polygon)

    # Plot Polygon
    fig = plt.figure() 
    ax = fig.gca() 
    BLUE = '#6699cc'
    GRAY = '#999999'

    patch = PolygonPatch(polygon, facecolor=GRAY, edgecolor=BLUE, alpha=0.5, zorder=2)
    ax.add_patch(patch)
    ax.scatter(x_pos, y_pos)
    ax.axis('scaled')
    plt.show()
    
    valid = scenic_polygon.containsPoint(position)
    print("Is the car in valid position? ==> ", valid)

    return valid


def checkHeading(scenario, obj_index, heading_angle, sampleNum = 100):    
    print("CheckPosition: Car"+str(obj_index) +" with heading angle of ", heading_angle)
    special_low_case, special_high_case = False, False
    bound1, bound2 = None, None
    headings = []
    threshold = 5
    valid = False
    
    if heading_angle < threshold:
        print("special_low_case")
        special_low_case = True 
        bound1 = 360 + (heading_angle - threshold)
        bound2 = heading_angle + threshold
        
    if heading_angle > 360 - threshold:
        print("special_high_case")
        special_high_case = True
        bound1 = heading_angle - threshold
        bound2 = heading_angle + threshold - 360
    
    upper, lower = False, False
    for i in range(sampleNum):
        scene = scenario.sampleForNuScenes()
        obj = scene.objects[obj_index]
        h = (obj.heading * 180 / math.pi + 360) % 360 # heading = [0, 360]
        headings.append(h)
        
        if not special_low_case and not special_high_case:
            if heading_angle - threshold <= h and h <= heading_angle:
                lower = True
            if heading_angle <= h and h <= heading_angle + threshold:
                upper = True
            
        if special_low_case:
            if heading_angle <= h and h <= bound2:
                lower = True
            if bound1 <= h or h <= heading_angle:
                upper = True
        
        if special_high_case:
            if bound1 <= h and h <= heading_angle:
                lower = True
            if heading_angle <= h or h <= bound2:
                upper = True
            
        if lower and upper:
            valid = True

    print("Is car's heading angle valid? ==> ", valid)
    plt.hist(x=headings, bins=72, color='#0504aa', alpha=0.7, rwidth=0.85)
    plt.show()
        
    return valid
        
    
    

In [ ]:
from scenic.core.scenarios import Scene
from scenic.core.vectors import Vector
from scenic.core.distributions import Constant
import math
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import copy

imgs_in_scenario = []
NUM_CAR = 1+1
# with open("/Users/edwardkim/Desktop/Scenic_bigdata_query/data/valid_numCar_trafficFlow_assumption/boston_seaport_valid_traffic_flow.pickle", 'rb') as f:
#         image_filenames = pickle.load(f)
# image_filenames.sort()
directory = "/Users/edwardkim/Desktop/nuScenes_data/samples/boston_seaport"
with open("/Users/edwardkim/Desktop/Scenic_bigdata_query/data/valid_numCar_trafficFlow_assumption/boston_seaport_valid_traffic_flow.pickle", 'rb') as f:
        image_filenames = pickle.load(f)

for img_name in image_filenames:
    img_map = query.img_filename_to_location[img_name]
    label = query.get_img_data(img_name)

    # for loop starts here over all images
    scenario_copy = scenario_list[img_map]
    print("------------------------------------------------------------------------------------")
    print(img_name)
    if isinstance(label, int):
        continue
    
    print("num Car: ", len(label['Vehicles']))

    # skip if the num car condition not met
    if (len(label['Vehicles']) >= NUM_CAR):
        req_valid = checkUserRequirements(scenario_copy, label, NUM_CAR)
        
        if req_valid[0]:
            print(label)
            img = mpimg.imread(os.path.join(directory, img_name))
            plt.imshow(img)
            plt.show()
            req_valid[1].show(zoom = True)
        else: 
            print("req_valid: ", req_valid)
            continue
        
#         if req_valid[0]:
#             req_valid[1].show(zoom = True)
        
        pos_heading_valid = checkPosHeading(scenario_copy, label, NUM_CAR)
        print("position/heading valid: " , pos_heading_valid)
        
        if req_valid[0] and pos_heading_valid:
            print("--------------------------THE SCENE IN THE SCENARIO-------------------------!")
            print(img_map)
            print(label)
            print("num Car: ", len(label['Vehicles']))
            imgs_in_scenario.append(img_name)


In [3]:
import os
import scenic
from scenic.simulators.gta.nusc_query_api import NuscQueryAPI

scenic_script = '../examples/gta/test_boston_seaport.sc'
scenario = scenic.scenarioFromFile(scenic_script)

Loading NuScenes tables for version v1.0-mini...
23 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
Done loading in 0.6 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.
boston-seaport


In [9]:
from scenic.simulators.gta.interface import CarModel
import itertools

def computeHeading(dictionary):
    updated_dictionary = {}
    
    # Filter Dictionary
    for key in dictionary.keys():
        if not isinstance(dictionary[key][0], CarModel):
            updated_dictionary[key] = dictionary[key]
    
    # Process the dictionary to a list to use itertools.product function
    total = []
    for key, value in updated_dictionary.items():
        l = []
        for v in value:
            l.append((key,v))
        total.append(l)
    
    # Find a Cartesian product of lists
    final = []
    for element in itertools.product(*total):
        final.append(element)
        
    output = {}
    for conditions in final:
        for condition in conditions:
            if condition[0] not in output:
                output[condition[0]] = condition[1]

    return final

In [10]:
print(scenario.dependencies[1].position._dependencies[0]._dependencies[0])

PointIn(IntersectionRegion((<scenic.core.regions.SectorRegion object at 0x154dce2e8>, <scenic.core.regions.PolylineRegion object at 0x1631287b8>)))


In [12]:
from scenic.core.vectors import Vector
# dictionary.clear()
# del dictionary
dictionary = {}
ego_car_pos= scenario.dependencies[0].position
ego_car_heading= scenario.dependencies[0].heading
nuscenes_ego_heading = label['EgoCar']['heading']

ego_car_pos.conditionTo(Vector(1724.481764481731, 787.6119609473986))
ego_car_heading.conditionTo(Constant(convert_deg_to_rad(nuscenes_ego_heading)))

car = scenario.dependencies[1]
car_pos = car.position
car_heading = car.heading
# car_pos.conditionTo(Vector(1724.481764481731, 787.6119609473986))
# dictionary = car_heading.findDist()
dictionary = car_pos.findDist()
print("dictionary: ",dictionary)

NameError: name 'label' is not defined

In [13]:
final = computeHeading(dictionary)
print("final: ", final)
print(len(final))

headings = []
for conditions in final:
    output = {}
    for condition in conditions:
        if condition[0] not in output:
            output[condition[0]] = condition[1]
    car_pos.condition_dists(output)
    s = car.sample()
    headings.append(s.position)
    
print(headings)

final:  [()]
1


NameError: name 'car_pos' is not defined

In [7]:
points = [(i.x, i.y) for i in headings]
x_pos, y_pos = [], []
for i in points:
    x_pos.append(i[0])
    y_pos.append(i[1])

max_x = x_pos.index(max(x_pos))
min_x = x_pos.index(min(x_pos))
print("max_x point: ", points[max_x])
print("min_x point: ", points[min_x])

max_y = y_pos.index(max(y_pos))
min_y = y_pos.index(min(y_pos))
mean_x_val = (x_pos[max_x] + x_pos[min_x])/2
mean_y_val = (y_pos[max_y] + y_pos[min_y])/2    

slope = (points[max_x][1] - points[min_x][1]) / (points[max_x][0] - points[min_x][0])
bias  = points[max_x][1] - slope * points[max_x][0] # b = y - slope * x

leftmost  = points[min_x]
rightmost = points[max_x]
upperhalf, bottomhalf = {},{}

for p in points:
    distance = computeDist(leftmost, p) + computeDist(rightmost, p)
    if p[1] - slope*p[0] > bias :
        upperhalf[p] = distance
    else:
        bottomhalf[p] = distance

upper_points = approximateContourPoints(upperhalf, x_pos[max_x], x_pos[min_x], reverse=False, coarseness = 11)
bottom_points = approximateContourPoints(bottomhalf, x_pos[max_x], x_pos[min_x], reverse=True, coarseness = 11)
print(upper_points)
print(bottom_points)

import matplotlib.pyplot as plt
plt.plot(x_pos, y_pos, 'ro')
plt.show()

NameError: name 'headings' is not defined

In [8]:
directory = "/Users/edwardkim/Desktop/nuScenes_data/samples/boston_seaport"
# image_filenames = [file for file in os.listdir(directory) if file.endswith('.jpg')]
img = "n008-2018-05-21-11-06-59-0400__CAM_FRONT__1526915427362465.jpg"
img_map = query.img_filename_to_location[img]
label = query.get_img_data(img)
traffic = label['traffic_flow']

KeyError: 'n008-2018-05-21-11-06-59-0400__CAM_FRONT__1526915427362465.jpg'

In [ ]:
from shapely.geometry import Polygon
from shapely.geometry import Point
import matplotlib.pyplot as plt

polygon1 = Polygon([(0, 0), (1, 1), (1, 0)])
polygon2 = Polygon([(0, 0), (0, 2), (1, 2), (1, 0)])
p = polygon1.intersection(polygon2)
# print(list(p.exterior.coords))

circle = Point(10,0).buffer(2.0)
intersection = circle.intersection(polygon2)
points = list(intersection.exterior.coords)
x_pos = [i[0] for i in points]
y_pos = [i[1] for i in points]
plt.plot(x_pos, y_pos, 'ro')
plt.show()
circle_points = set(circle.exterior.coords)
print(intersection.is_empty)
print(len(circle_points))

In [ ]:
from scenic.core.regions import SectorRegion, RectangularRegion
from scenic.core.vectors import Vector
import math

region = SectorRegion(Vector(0,0), 1, 0, math.pi/2, resolution = 65)
rectangle = RectangularRegion(Vector(0,0), 0, 2, 4)
points = rectangle.corners
x = [i.x for i in points]
y = [i.y for i in points]
plt.plot(x, y, 'ro')
plt.show()
print(points)

In [ ]:
from scenic.core.regions import *

def findRegionContourPoints(pos, dictionary={}, depth=0):
    for child in pos._conditioned._dependencies:
        print("child: ", child)
        
        if isinstance(child, PointInRegionDistribution):
            print("PointInDistribution")
            dictionary[child] = child.findContourPoints()
            continue
        
        if child._conditioned._dependencies != ():
            print("traverse down!")
            findRegionContourPoints(child, dictionary, depth+1)
        
    return dictionary



In [ ]:
from scenic.core.distributions import Constant
ego_car_pos = scenario.dependencies[0].position
ego_car_heading = scenario.dependencies[0].heading
nuscenes_ego_heading = label['EgoCar']['heading']

ego_car_pos.conditionTo(Vector(1724.481764481731, 787.6119609473986))
ego_car_heading.conditionTo(Constant(convert_deg_to_rad(nuscenes_ego_heading)))

car = scenario.dependencies[1]
car_pos = car.position
car_heading = car.heading

In [ ]:
# print(car_pos)
dic = findRegionContourPoints(car_pos)

In [ ]:
for k in dic.keys():
    print(dic[k])